In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import convolve2d as conv
from scipy.signal import correlate2d as corr
from FromScratchModule.NNet import Base_Layer

In [2]:
a = np.random.randint(0,3,size=[2,5,5])  # (C, L, W)

In [3]:
class Convolutional(Base_Layer):
    def __init__(self, channels_in, channels_out, filter_size, learn_rate=1e-3, activation_func=lambda x: x*(x>0), padding='same'):
        self.filters = np.random.rand(channels_out, channels_in, filter_size, filter_size)
        self.activation_func = activation_func
        self.padding = padding
        self.learn_rate = learn_rate
        
    def Forward(self, x):
        out = [np.sum([corr(x[channel], f[channel], mode='same') for channel in range(x.shape[0])], axis=0)
               for f in self.filters]

        out = np.stack(out)
        return out
    
    def Backward(self, z_in, y):
        h_out = np.stack([np.sum([corr(z_in[channel], f[channel], mode='same') for channel in range(z_in.shape[0])], axis=0)
               for f in self.filters])
        
        z_out = self.activation_func(h_out)
        
        grad_after = self.after.Backward(z_out, y)
        
        grad_h = grad_after * self.activation_func.D(h)
        
        for f in range(self.filters.shape[0]):
            grad_inner = []
            for channel in range(self.filters.shape[1]):
                grad_inner.append(correlate2d(grad_h[f], z_in[channel], mode='same'))
            grad_f.append(np.stack(grad_inner))
        grad_f = np.stack(grad_f)
        
        grad_z_in = []
        for channel in range(self.filters.shape[1]):
            grad_inner = 0
            for channel in range(self.filters.shape[0]):
                grad_inner += convolve2d(grad_h[f], self.filters[f, channel], mode='same')
            grad_z_in.append(grad_inner)
        grad_z_in = np.stack(grad_z_in)
        
        self.filters -= grad_f*self.learn_rate

In [4]:
model = Convolutional(2, 7, 3)

In [13]:
z = model.Forward(a)
z.shape

(7, 5, 5)

In [66]:
class MaxPoolingLayer(Base_Layer):
    def __init__(self, factor):
        self.factor = factor
            
        
    def Forward(self, z_in):
        out_height = int(np.ceil(float(z_in.shape[1]) * self.factor))
        out_width = int(np.ceil(float(z_in.shape[2]) * self.factor))

        pool_height = np.maximum(1, int(z_in.shape[1] * (1 - self.factor)))
        pool_width = np.maximum(1, int(z_in.shape[2] * (1 - self.factor)))
        
        z_out = np.zeros([z_in.shape[0], out_height, out_width])
        
        print(z_in)

        for i,mat in enumerate(z_in):
            print(mat)
            for j in range(out_width - 1):
                for k in range(out_height - 1):
                    p = mat[k*pool_height:(k+1)*pool_height, j*pool_width:(j+1)*pool_width]
                    print((i,j,k), p)
                    z_out[i,j,k] = p.max()
                p = mat[(out_height - 1)*pool_height:, j*pool_width:(j+1)*pool_width]
                print((i,j,k), p)
                z_out[i, j, -1] = p.max()
            
            for k in range(out_height - 1):
                p = mat[k*pool_height:(k+1)*pool_height, (out_width - 1)*pool_width:]
                print((i,j,k), p)
                z_out[i,-1,k] = p.max()
            p = mat[(out_height - 1)*pool_height:, (out_width - 1)*pool_width:]
            print((i,j,k), p)
            z_out[i, -1, -1] = p.max()
                    
        return z_out.T
    
    
    def Backward(self, z_in, y):
        pool_height = int(np.floor(z_in.shape[1] * self.factor))
        pool_width = int(np.floor(z_in.shape[2] * self.factor))
        
        grad_height = int(np.ceil(float(z_in.shape[1]) / pool_height))
        grad_width = int(np.ceil(float(z_in.shape[2]) / pool_width))
        grad_z_in = np.zeros([x.shape[0], grad_height, grad_width])
        
        grad_after = self.after.Backward(z_in, y)
        
        for i,mat in enumerate(z_in):
            for j in range(grad_width - 1):
                for k in range(grad_height - 1):
                    ind = np.argmax(mat[k*pool_height:(k+1)*pool_height, j*pool_width:(j+1)*pool_width])
                    grad_z_in[i,j,k] = grad_after[ind]
                ind = np.argmax(mat[(grad_height - 1)*pool_height:, j*pool_width:(j+1)*pool_width])
                grad_after[i, j, -1] = grad_after[ind]
                
            for k in range(grad_height - 1):
                ind = np.argmax(mat[k*pool_height:(k+1)*pool_height, (j+1)*pool_width:])
                grad_z_in[i,-1,k] = grad_after[ind]
            ind = np.argmax(mat[(grad_height - 1)*pool_height:, (grad_width - 1)*pool_width:])
            grad_z_in[i, -1, -1] = grad_after[ind]
                    
        return grad_z_in
        
                  
### Use np.array_split

In [72]:
pool = MaxPoolingLayer(0.5)
pool.Forward(a)

AttributeError: 'MaxPoolingLayer' object has no attribute 'Forward'

In [64]:
np.reshape(a[0], (1,) + a[0].shape)

array([[[1, 2, 2, 1, 2],
        [0, 2, 1, 2, 2],
        [2, 0, 0, 1, 0],
        [0, 2, 1, 0, 1],
        [2, 0, 2, 2, 1]]])

In [60]:
a[0][:2, :2]

array([[1, 2],
       [0, 2]])

In [42]:
np.maximum(a)

ValueError: invalid number of arguments